# Final Exam (Linear Programming and Approximation Algorithms)

This is a take home final exam. You have __30 hours__ to tackle three problems where you will be expected to come up with algorithms for solving two problems, coding up the solution and passing the test cases. 

  - The time limit is strict: submissions or modifications of earlier submissions after the time limit has elapsed will not be accepted and receive no points.

This exam is open book and notes. You are allowed to consult any course material presented in this class. However, seeking external help (searching the internet, asking others or asking a generative AI system) for solving the problems online is _strictly forbidden_. You may however consult online documentation on the python language or the pulp LP solver.

To be clear: 
  - It is OK to consult online resources on python programming. Eg., you forgot how to implement a class in Python or you need to consult the documentation for PuLP, then it is OK to search online.
     - Using generative AI for basic programming help is fine thought not encouraged for an exam.
  - It is forbidden to ask for any kind of online help for solving the algorithmic problems.
     - Using generative AI for help with solving the problem is forbidden.
     - Posting these problems on online forums is forbidden.
     - Searching online for solutions is forbidden.
     - Asking a classmate or friend for help is forbidden.
 
 

## Problem 1 (25 points)

A logistics company ships natural gas over trains between different places. The places form the vertices of an undirected graph and each (bi-directional) edge between two vertices denotes a rail line over which fuel can be transported by train in either direction.  We associate with each edge a cost/unit that specifies how much it costs to send one unit of oil between the two end points (in either direction). Some of the vertices are oil producing vertices where there is a net supply of oil whereas the other vertices are demand vertices where there is a net demand for oil. We will write demand as negative supply.

### Example

Here is an example involving five cities: $\{ Houston, Chicago, Baltimore, Philadelphia, Charleston \}$ and the edges are shown below along with shipping cost per unit along each edge (in red).

<img src="attachment:cities-graph.jpeg" width="60%"></img>

The following table summarizes the net supply (or demand) in metric tonnes for each city.

$$\begin{array}{l l}
\hline
\text{City} & \text{Net Supply} \\ 
\hline 
Houston & 100 \\ 
Chicago & -55 \\ 
Baltimore & 35 \\ 
Philadelphia & -40 \\ 
Charleston & - 25 \\ 
\hline
\end{array}$$

Notice that in Houston and Baltimore, there is a net supply whereas Chicago, Philadelphia and Charleston have net demands (negative supplies). 

We wish to find out how much oil to transport along each of the rail lines (edges of the graph)  so that the following constraints are met and the total transportation cost is minimized.

  - For each city with a net oil demand, the total inflow of oil into the city minus the total outflow of oil from the city must be equal to the total demand of the city.
  - For each city with a net oil supply, the total outflow of oil from the city minus the total inflow into the city must be _less than or equal_ to the net supply available for the city. 
  



For instance, consider the following shipping schedule:


<img src="attachment:cities-shipping-schedule.jpeg" width="60%"> </img>

We ship $95$ units of oil from Houston to Chicago, $40$ units from Chicago to Philly, and $25$ units of oil from Baltimore to Charleston. 

Does it satisfy all constraints?
  - At Houston there is a net supply of $100$. The total outflow is $95$ and total inflow is $0$. We have $95 \leq 100$.
  - At Chicago, there is a net demand of $55$. Total inflow is $95$ and total outflow is $40$. Thus $95 - 40 = 55$.
  - Similarly, verify that the constraints are satisfied at all the other cities.
  
The total transportation cost is $ 95 * 5 + 40 * 4 + 25 * 8 = 835$. Can we do better than this in terms of cost?

Formulate a linear programming problem to solve the transportation problem. You are given as input: 
  - Graph with vertices $V$ and edges $E$. Each _undirected_ edge $(i,j)$ can be regarded as two edges $(i,j)$ and $(j,i)$ in opposite directions, if it is easier for your solution. 
  - A cost $c: E \mapsto \mathbb{R}$ associated with each edge such that $c(e) \geq 0$ and is the cost of transporting one unit of oil along the edge.
  - A map $s: V \mapsto \mathbb{R}$ mapping each vertex $v$ to net supply $s(v)$. (note that if $s(v) \leq 0$ then it is considered a demand).
  

We suggest following steps on pencil/paper to formulate the problem.
  - (a) identify all the decision variables,
  - (b) write down the constraints, and
  - (c) write down the objective function.
  
Complete the python function `calculateOptimalPlan` that takes inputs:
  - `n`: number of vertices which are numbered $0, \ldots, n-1$;
  - `edge_list`: a list of undirected edges $(i,j,c)$ between vertices wherein $c \geq 0$ is the cost of flow along the edge; 
  - `supplies`: a list of size $n$ where `supplies[j]` is the supply (or demand if negative) at the jth vertex.
  
Your function must return a dictionary that maps edges `(i,j)` to the flow along the edge in the direction $i \rightarrow j$.
  - All flows  must be non-negative.
  - If you specify a flow from $j$ to $i$, then your dictionary must have the key `(j,i)` mapped to the non-negative flow from $j$ to $i$.
  - If an edge is not present in the dictionary, we will take the flow along it to be zero.

In [1]:
from pulp import *
def calculateOptimalPlan(n, edge_list, supplies, debug=False):
    assert n >= 1
    assert all( 0 <= i < n and 0 <= j < n and i != j and c >= 0 for (i,j,c) in edge_list)
    assert len(supplies) == n
    # TODO: Formulate the LP for optimal transportation plan and return the solution as a dictionary
    #       from edges (i,j) to flow from i to j.
    #       If an edge is not present in the dictionary, we will take its flow to be zero.
    # your code here
    # Create a set of existing edges for quick lookup
    existing_edges = set((i, j) for i, j, _ in edge_list if i != j)
    existing_edge_list = []

    for i, j, c in edge_list:
        existing_edges.add((j, i))  # Ensure reverse direction exists as well
        existing_edge_list.append((i,j,c))
        existing_edge_list.append((j,i,c))
       
    # Create a LP problem
    prob = LpProblem("OilTransportation", LpMinimize)

    # Define variables for flow along existing edges
    flow = LpVariable.dicts("Flow", existing_edges, lowBound=0)

    # Objective function: Minimize total transportation cost for existing edges
    prob += lpSum(flow[(i, j)] * c for (i, j, c) in existing_edge_list if (i, j) in existing_edges)

    # Constraints for existing edges
    for i in range(n):
        total_inflow = lpSum(flow[(j, i)] for j in range(n) if j != i and (j, i) in existing_edges)
        total_outflow = lpSum(flow[(i, j)] for j in range(n) if j != i and (i, j) in existing_edges)
        
        if supplies[i] >= 0:  # Net supply available
            prob += total_outflow - total_inflow <= supplies[i]  # Constraint #2
        else:  # Net demand
            prob += total_inflow - total_outflow == - supplies[i] # Constraint #1

    # Solve the problem
    prob.solve()

    # Extract the optimal solution for existing edges
    optimal_flow = {(i, j): flow[(i, j)].varValue for (i, j) in existing_edges if flow[(i, j)].varValue != 0}

    # Return the dictionary mapping existing edges to non-zero flow
    return optimal_flow

In [2]:
def test_solution(n, edge_list, supplies, solution_map, expected_cost):
    cost = 0
    outflows = [0]*n
    inflows = [0]*n
    for (i,j,c) in edge_list:
        if (i,j) in solution_map: 
            flow = solution_map[(i,j)]
            cost += c * flow
            assert flow >= 0, f'flow on edge {(i,j)} is negative --> {flow}'
            outflows[i] += flow 
            inflows[j] += flow
        elif (j,i) in solution_map:
            flow = solution_map[(j,i)]
            cost += c * flow
            assert flow >= 0, f'flow on edge {(j,i)} in negative --> {flow}'
            outflows[j] += flow
            inflows[i] += flow 
    for (i, s) in enumerate(supplies):
        if s > 0:
            assert outflows[i]  - inflows[i] <= s, f'Vertex {i} constraint violated: total outflow = {outflows[i]} inflow = {inflows[i]}, supply = {s}'
        else:
            assert abs(inflows[i]-outflows[i] + s) <= 1E-2,f'Vertex{i} constraint violated: inflow = {inflows[i]} outflow={outflows[i]}, demand = {-s}'
    if expected_cost != None:
        assert abs(expected_cost - cost) <= 1E-02, f'Expected cost: {expected_cost}, your algorithm returned: {cost}'
    print('Test Passed!')

n = 5
edge_list = [
    (0,1, 5), (0, 3, 3), (0, 4, 4),
    (1,2, 9), (1,4, 6),
    (2,3,8),
    (3,4,7)
]
supplies = [-55, 100, -25, 35, -40]
sol_map = calculateOptimalPlan(n, edge_list, supplies, debug=True)
test_solution(n, edge_list, supplies, sol_map,670)


print('5 points!')

Test Passed!
5 points!


In [3]:
n = 10
edge_list = [
    (0, 1, 5),
    (0, 2, 4),
    (0, 3, 7),
    (0, 4, 3),
    (0, 5, 9),
    (0, 8, 6),
    (0, 9, 5),
    (1, 2, 3),
    (2, 4, 9),
    (2, 7, 8),
    (2, 8, 7),
    (2, 6, 5),
    (3, 4, 6),
    (3, 5, 7),
    (3, 6, 4),
    (3, 7, 8),
    (3, 8, 3),
    (3, 9, 5),
    (4, 8, 5),
    (5, 7, 8),
    (6, 8, 2),
    (7, 8, 3),
    (7, 9, 6),
    (8, 9, 10)
]
supplies=[
    20,
    30,
    -30,
    -40,
    10,
    15,
    20,
    -35,
    40,
    -30
]

sol_map = calculateOptimalPlan(n, edge_list, supplies, debug=True)
test_solution(n, edge_list, supplies, sol_map,575)
print('5 points')

Test Passed!
5 points


In [4]:
from random import randint,seed
def gen_random_test(n, num_edges):
    assert n >= 1
    edge_list = [(i,i+1, randint(2,10)) for i in range(n-1)]
    
    while len(edge_list) < num_edges:
        i = randint(0, n-1)
        j = randint(0, n-1)
        (i,j) = (min(i,j), max(i,j))
        if i == j: 
            continue 
        if any( ihat == i and jhat == j for (ihat, jhat, _) in edge_list):
            continue
        c = randint(2, 10)
        edge_list.append((i,j,c))
    tot = 0
    supplies=[]
    for i in range(n-1):
        si = randint(-100, 100) 
        supplies.append(si)
        tot = tot + si

    if tot <= 0:
        supplies.append(-tot)
    else:
        supplies.append(randint(1-tot, 0))
    
    return (n, edge_list, supplies)

seed(10001)
(n, edge_list,supplies) = gen_random_test(50, 100)
print(edge_list)
sol_map = calculateOptimalPlan(n, edge_list, supplies, debug=True)
test_solution(n, edge_list, supplies, sol_map,None)

(n, edge_list,supplies) = gen_random_test(45, 50)
print(edge_list)
sol_map = calculateOptimalPlan(n, edge_list, supplies, debug=True)
test_solution(n, edge_list, supplies, sol_map,None)


(n, edge_list,supplies) = gen_random_test(15,80)
print(edge_list)
sol_map = calculateOptimalPlan(n, edge_list, supplies, debug=True)
test_solution(n, edge_list, supplies, sol_map,None)

print('15 points!')

[(0, 1, 7), (1, 2, 3), (2, 3, 3), (3, 4, 8), (4, 5, 8), (5, 6, 3), (6, 7, 10), (7, 8, 6), (8, 9, 4), (9, 10, 3), (10, 11, 6), (11, 12, 2), (12, 13, 9), (13, 14, 6), (14, 15, 4), (15, 16, 5), (16, 17, 7), (17, 18, 6), (18, 19, 8), (19, 20, 9), (20, 21, 7), (21, 22, 9), (22, 23, 6), (23, 24, 8), (24, 25, 2), (25, 26, 9), (26, 27, 3), (27, 28, 10), (28, 29, 8), (29, 30, 7), (30, 31, 3), (31, 32, 7), (32, 33, 2), (33, 34, 3), (34, 35, 5), (35, 36, 7), (36, 37, 3), (37, 38, 2), (38, 39, 9), (39, 40, 7), (40, 41, 6), (41, 42, 2), (42, 43, 10), (43, 44, 4), (44, 45, 9), (45, 46, 7), (46, 47, 5), (47, 48, 9), (48, 49, 3), (42, 45, 9), (23, 48, 2), (3, 31, 8), (21, 24, 3), (13, 37, 4), (14, 30, 8), (28, 48, 8), (7, 41, 3), (20, 22, 7), (7, 23, 2), (7, 18, 6), (15, 36, 4), (18, 21, 2), (45, 48, 6), (15, 20, 9), (11, 38, 2), (15, 33, 8), (22, 32, 7), (8, 42, 6), (3, 19, 10), (20, 29, 9), (9, 30, 7), (6, 46, 3), (2, 20, 5), (8, 31, 8), (40, 46, 8), (4, 22, 3), (23, 38, 3), (7, 49, 8), (16, 28, 2),

### Problem 2 : 40 points

In this problem, you are given a set of inequality constraints $I_1, \ldots, I_m$, involving variables $(x_1, \ldots, x_n)$. 

For example, consider six inequalities shown below involving $(x_1, x_2)$:

$$ \begin{array}{rll}
 x_1 - x_2 & \leq -5 & \leftarrow I_1 \\ 
 x_1 + 2 x_2 & \leq 3 & \leftarrow I_2 \\ 
 x_1 & \geq 4  & \leftarrow I_3\\ 
 x_1 & \leq -2  & \leftarrow I_4\\ 
 x_2 & \geq 3  & \leftarrow I_5\\ 
 x_2 & \leq -1  & \leftarrow I_6\\ 
 \end{array}$$
 
 
 Further, we give you a possible range of values for each variable: $x_i \in [\ell_i, u_i]$ for limits $\ell_i, u_i$ and $i = 1, \ldots, n$.
 
In our example:  $x_1 \in [-10, 10], x_2 \in [-10, 10]$.
 
 Your goal is to find values for $(x_1, \ldots, x_n)$ wherein each $x_i$ lies within its bounds $[\ell_i, u_i]$ and at the same time satisfies as many of the inequalities above as possible.
 
For instance if we set $x_1 = -2, x_2 = -1$ in the example above, we satisfy $3$ out of the six inequalities:

$$ \begin{array}{rl}
\text{inequality} & \text{satisfied} \\ 
\hline
 x_1 - x_2  \leq -5 & \text{N}\\ 
 x_1 + 2 x_2  \leq 3 & \text{Y} \\ 
 x_1 \geq 4  &\text{N}\\ 
 x_1  \leq -2 & \text{Y} \\ 
 x_2  \geq 3 & \text{N} \\ 
 x_2  \leq -1 & \text{Y}\\ 
 \end{array}$$
 
 Can we satisfy more than $3$ inequalities by choosing some values $(x_1, x_2)$ in the range?
 
 **Inputs:** $n$ variables and $m$ inequalities.
 - Each inequality $I_j$ is of the form:
 $$I_j:  c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j \,.$$
 - Ranges $[\ell_i, u_i]$ for variable $x_i$ where $\ell_i \leq u_i$.
Note that the variables $x_1, \ldots, x_n$ can take on real-values in the range.

**Output** A point $(x_1, \ldots, x_n)$ wherein each $x_i \in [\ell_i, u_i]$ and the number of satisfied inequalities is maximized.
 
 ## (A) (Mixed) Integer Linear Programming Formulation (20 points)
 
 We will guide you through a mixed integer programming formulation for the problem.  
 - For each each inequality $I_j$ given to us, a binary decision variable $w_j \in \{0, 1\}$ which is 1 if the inequality is to be satisfied and  0 otherwise.

(i) Write down the objective in terms of $w_1, \ldots, w_m$. Write your answer in the space below. It is not graded and we provide selected answers at the end as hints.


**YOUR ANSWER HERE**

$$\max\ w_1 + \cdots + w_m $$


(ii) Consider the inequality  $$I_j:  c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j \,.$$

Given that each $x_i \in [\ell_i, u_i]$ are the bounds for each variable $x_i$,  let $\mathsf{upperBoundLHS}(I_j)$ denote the maximum value that the LHS Expression $c_{j1} x_1 + \cdots + c_{jn} x_n$ takes in the
domain $x_1 \in [\ell_1, u_1], \ldots, x_n \in [\ell_n, u_n]$.

As an example, suppose $x_1 \in [-1, 1]$ and $x_2 \in [-3, 2]$ then an upper bound on the expression $2 x_1 - 4 x_2$ is given by $ 2 \times 1 - 4 \times (-3) = 14$.

Write an expression for $\mathsf{upperBoundLHS}(c_1 x_1 + \cdots + c_n x_n \leq d)$ as a function of 
$c_1, \ldots, c_n, d, \ell_1, u_1, \ldots, \ell_n, u_n$.


**YOUR ANSWER HERE**

### 2(A) part (ii)

$ \sum_{i=1}^n c_i b_i $ where $b_i = \begin{cases} \ell_i & \text{if}\ c_i < 0 \\ 
u_i & \text{if}\  c_i \geq 0\\
\end{cases}$.


(iii) We are now ready for encoding the problem of maximizing the number of satisfied inequalities as an mixed integer linear programming involving decision variables  $x_1, \ldots, x_n \in \mathbb{R}$ and $w_1, \ldots, w_m \in \{0, 1\}$.

For each inequality $I_j: c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j$, we convert it into the following 
inequality
$$ \hat{I}_j:\ c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j w_j + M_j (1 - w_j) $$
where $M_j = \mathsf{upperBoundLHS}(I_j)$.

Prove that $\hat{I}_j$ is the same as $I_j$ when $w_j = 1$ and $\hat{I}_j$ is implied by the other constraints 
when $w_j = 0$. Write down the argument below (not graded).


**YOUR ANSWER HERE**

### 2(A) part (iii)
When $w_j = 1$, we have $\hat{I}_j$ is the inequality

$$ c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j \times 1 + M_j \times 0 = d_j $$

Therefore, $\hat{I}_j$ and $I_j$ coincide.

However, when $w_j = 0$, we have  $\hat{I}_j$ is the inequality

$$ c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j \times 0 + M_j \times 1 = M_j $$
However, given that $x_i \in [\ell_i, u_i]$, and $M_j = \mathsf{upperBoundLHS}(I_j)$,
it automatically implies that that $\hat{I}_j$ is satisfied trivially by whatever value of $(x_1, \ldots, x_n)$ we choose.

Complete the formulation of the optimizaition problem and implement the function `solveForMaximumInequalitySatisfaction` with the following arguments
  - `n` the number of variables
  - `m` the number of inequalities
  - `c_matrix`: a list of list of coefficients  of the LHS of inequalities
  $$ \left[ \begin{array}{c}
  [ c_{11}, \ldots, c_{1n}], \\ 
  [ c_{21}, \ldots, c_{2n}], \\  
  \cdots \\
  [ c_{m1}, \ldots, c_{mn}] \\
  \end{array}\right]$$
  Please note python indexes starting from 0.
 - `d_values`: a list of RHS coefficients: $[ d_1, \ldots, d_m ] $
 - `bounds: `: a list of pairs $[(\ell_1, u_1), \ldots, (\ell_n, u_n)]$ for each variable.
 
 Your function should return a pair:  $(k, [x_1, \ldots, x_n ] )$
   - The number of inequalities satisfied by your optimal solution ($k$)
   - A list denoting the values of $x_1, \ldots, x_n$ that satisfy the $k$ inequalities.
   
Please use the pulp solver.

In [1]:
from pulp import *

# Here is a useful function to implement the LHS upper bound that we need for the encoding
def lhsUpperBound(c_list, bounds):
    n = len(c_list)
    assert len(bounds) == n 
    upper_bnd = sum([(cj*lj) if cj < 0 else cj*uj for (cj, (lj, uj)) in zip(c_list, bounds) ])
    return upper_bnd

def solveForMaximumInequalitySatisfaction(n, m, c_matrix, d_values, bounds):
    # always check pre-conditions: saves so much time later
    assert len(c_matrix) == m
    assert all(len(c_list) == n for c_list in c_matrix)
    assert len(d_values) == m
    assert len(bounds) == n
    assert all (lj <= uj for (lj, uj) in bounds)
    ## TODO: set up and solve the problem for satisfying the maximum number of inequalities
    # your code here
    prob = LpProblem("Maximize_Inequality_Satisfaction", LpMaximize)

    # Define variables
    x = [LpVariable(f"x{i}", lowBound=b[0], upBound=b[1]) for i, b in enumerate(bounds)]
    w = [LpVariable(f"w{i}", cat='Binary') for i in range(m)]

    # Add objective: maximize the number of inequalities satisfied
    prob += lpSum(w), "Objective"

    # Add constraints
    for j in range(m):
        prob += lpSum(c_matrix[j][i] * x[i] for i in range(n)) <= (d_values[j] * w[j] + lhsUpperBound(c_matrix[j], bounds)*(1-w[j]))

    # Solve the problem
    prob.solve()

    # Collect the solution
    k = int(sum(w[i].varValue for i in range(m)))
    solution_values = [x[i].varValue for i in range(n)]

    return k, solution_values
    
    

In [2]:
def testSolution(n, m, c_matrix, d_values, bounds, x_values, k_expected):
    # always check pre-conditions: saves so much time later
    assert len(c_matrix) == m
    assert all(len(c_list) == n for c_list in c_matrix)
    assert len(d_values) == m
    assert len(bounds) == n
    assert all (lj <= uj for (lj, uj) in bounds)
    assert len(x_values) == n
    assert 0 <= k_expected <= m
    # check solution within bounds
    for i in range(n):
        (lb, ub) = bounds[i]
        assert lb <= x_values[i] <= ub, f'x_{i} fails to be within its bounds {[lb, ub]}'
    # Check how many inequalities satisfied
    num_ineqs = 0
    
    for (c_list, d) in zip(c_matrix, d_values):
        if sum([cj * xj for (cj, xj) in zip(c_list,x_values )]) <= d+1E-3:
            num_ineqs = num_ineqs + 1
    assert num_ineqs == k_expected, f' Expected number of inequalities to be sat: {k_expected} your solution satisfies: {num_ineqs} inequalities '
    print('Test Passed')
    return 
        
        
        
        

# x_1 - x_2 & \leq -5 & \leftarrow I_1 \\ 
# x_1 + 2 x_2 & \leq 3 & \leftarrow I_2 \\ 
# x_1 & \geq 4  & \leftarrow I_3\\ 
# x_1 & \leq -2  & \leftarrow I_4\\ 
# x_2 & \geq 3  & \leftarrow I_5\\ 
# x_2 & \leq -1  & \leftarrow I_6\\ 

n = 2
m = 6
c_matrix = [
    [1, -1],
    [1, 2],
    [-1, 0],
    [1, 0],
    [-1, 0],
    [1, 0]
]

d_list = [
    -5, 3, -4, -2, -3, -1
]

bounds = [(-10, 10), (-10, 10)]

(k, x_values) = solveForMaximumInequalitySatisfaction(n, m, c_matrix, d_list, bounds)
testSolution(n, m, c_matrix, d_list, bounds, x_values, 4)
print('5 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/cb8b3a9de7134b2a8d1d56ad48d5da48-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/cb8b3a9de7134b2a8d1d56ad48d5da48-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 44 RHS
At line 51 BOUNDS
At line 62 ENDATA
Problem MODEL has 6 rows, 8 columns and 14 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5.17183 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 8 columns (6 integer (6 of which binary)) and 16 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial st

In [3]:

n = 3
m = 12
c_matrix = [
    [1, -1, 0],
    [1, 2, 0],
    [-1, 0, 1],
    [1, 0, 0],
    [-1, 0, 0],
    [1, 0, 0],
    [1, 0, -1],
    [0, 2, 1],
    [-1, 1, 1],
    [1, 1, 1],
    [-1, 1, 1],
    [1, 1, 1],
]

d_list = [
    -5, 3, -4, -2, -3, -1,
    -5, 3, -4, -2, -3, -1
]

bounds = [(-10, 10), (-10, 10), (-12, 12)]

(k, x_values) = solveForMaximumInequalitySatisfaction(n, m, c_matrix, d_list, bounds)
testSolution(n, m, c_matrix, d_list, bounds, x_values, 10)
print('7 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/f618132784c2436fae734503c1cbdc3a-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/f618132784c2436fae734503c1cbdc3a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 91 RHS
At line 104 BOUNDS
At line 123 ENDATA
Problem MODEL has 12 rows, 15 columns and 37 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 11.1787 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 12 rows, 15 columns (12 integer (12 of which binary)) and 38 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I In

In [4]:
n = 5
m = 24
c_matrix = [
    [1, -1, 0, 1, -1],
    [1, 2, 0, 0, 2],
    [-1, 0, 1, 1, 1],
    [1, 0, 0, 0, -1],
    [-1, 0, 0, -1, -1],
    [1, 0, 0, 1, 1],
    [1, 0, -1, 1, 0],
    [0, 2, 1, 0, 2],
    [-1, 1, 1, -1, 0],
    [1, 1, 1, 0, 1],
    [-1, 1, 1, 0, 0],
    [1, 1, 1, 1, 0],
    [-1, 1, 0, 1, -1],
    [1, -2, 0, 0, -2],
    [1, 0, 1, -1, -1],
    [1, 0, 1, 0, 1],
    [-1, 0, 0, 1, 1],
    [-1, 0, 0, 1, 1],
    [1, -1, 1, 1, 1],
    [0, -2, -1, 0, 2],
    [-1, -1, -1, -1, 0],
    [-1, 1, -1, 0, 1],
    [1, 0, 0, 1, 0],
    [-1, 0, -1, 0, -1],
]

d_list = [
    -5, 3, -4, -2, -3, -1,
    -5, 3, -4, -2, -3, -1,
     5, -3, 4, 2, 3, 1,
    5, -3, 4, 2, 3, 1,
    
]

bounds = [(-10, 10), (-10, 10), (-12, 12), (-1, 3), (3, 6)]

(k, x_values) = solveForMaximumInequalitySatisfaction(n, m, c_matrix, d_list, bounds)
testSolution(n, m, c_matrix, d_list, bounds, x_values, 18)
print('8 points')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/acd227122c1b44bbb6295a6369b06d31-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/acd227122c1b44bbb6295a6369b06d31-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 207 RHS
At line 232 BOUNDS
At line 267 ENDATA
Problem MODEL has 24 rows, 29 columns and 105 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 22.9033 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 24 rows, 29 columns (24 integer (24 of which binary)) and 107 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I

## Part (B) Design a Factor-2 Approximation Algorithm (20 points)


We will design a factor-2 approximation algorithm. To do so, first consider the case when there is just one variable in our problem.

#### Example

$$ \begin{array}{l}
x & \leq 5 \\ 
-x & \leq -4 \\ 
x & \leq 3 \\ 
x & \leq 15 \\ 
-x & \leq -6 \\ 
-x & \leq -8  \\
-x & \leq -3\\
\end{array}$$


Let us consider $m$ inequalities involving just one variable $x$. 

For convenience, we will drop  the upper/lower bounds. In other words, the bounds are simply 
$$x \in (-\infty, \infty)\,.$$


(i) Design an algorithm that discovers a value of $x$ guaranteed to satisfy $\lceil \frac{m}{2} \rceil$ or more inequalities. 

**Hint** Try satisfying all the inequalities of the form $x \leq d_j$ at once or alternatively, try satisfying all inequalities of the form $ - x \leq d_j$  at once. 



Write down your approach below (not graded).


YOUR ANSWER HERE

Using the algorithm for $n = 1$ variables, implement an algorithm that achieves a factor-2 approximation for inequalities with arbitrary $n$.

Specifically, given a system of $m$ inequalities over $x_1, \ldots, x_n$, provide an algorithm that finds a solution $(x_1, \ldots, x_n)$ that satisfies $\geq \frac{m}{2}$ inequalities. 

Once again, we drop the upper/lower bounds on the variables. I.e, $x_i \in (-\infty, \infty)$.

**Hint** Generate $n$ random numbers $r_1, \ldots, r_n$ in some range $[-1,1]$ and simply substitute: 
$$ x_1 = r_1 x, x_2 = r_2 x, \ldots, x_n  = r_n x$$
where  $x$ is a new unknown variable. 
Use this trick to reduce the constraints involving $n$ variables to a single variable $x$; and apply previous result. 

Implement a function `computeApproximateSolution` with inputs
 - `n`: number of variables.
 - `m`: number of inequalities
 - `c_matrix`: a list of list of coefficients  of the LHS of inequalities
  $$ \left[ \begin{array}{c}
  [ c_{11}, \ldots, c_{1n}], \\ 
  [ c_{21}, \ldots, c_{2n}], \\  
  \cdots \\
  [ c_{m1}, \ldots, c_{mn}] \\
  \end{array}\right]$$
  Please note python indexes starting from 0.
 - `d_values`: a list of RHS coefficients: $[ d_1, \ldots, d_m ] $
 
 Your function should return a pair:  $(k, [x_1, \ldots, x_n ] )$
   - The number of inequalities satisfied by your optimal solution ($k$)
   - A list denoting the values of $x_1, \ldots, x_n$ that satisfy the $k$ inequalities.
   
 Also for this problem, we require $k \geq \frac{m}{2}$.
 
 **Note:** The test cases below will run for large values of $n, m$. If your implementation uses an integer linear programming solver, it may not finish within the time budget of 2 minutes allocated for grading the notebook.

In [555]:
from random import uniform 
import numpy as np

def computeApproximateSolution(n, m, c_matrix, d_values):
    assert n >= 1
    assert len(c_matrix) == m
    assert all(len(c_list) == n for c_list in c_matrix)
    assert len(d_values) == m
    r_values = [uniform(-1, 1) for i in range(n)]
    # your code here
    seed(100001)
    k = 0
    solution_values = []
    attemps = 0
    num_ineqs = 0
    while (num_ineqs < (m / 2)):
        r_values = [uniform(-1, 1) for i in range(n)]

        # Step 2: Reformulate the problem based on x instead of xi
        # Step 3: Compute the matrix product
        coefs = [sum(c_matrix[i][j] * r_values[j] for j in range(n)) for i in range(m)]
        
        # Reformulate the inequalities
        coefs_abs = [abs(coef) for coef in coefs]
        normalized_coefs = [coef_val / abs_coef for coef_val, abs_coef in zip(coefs, coefs_abs)]
        normalized_d_values = [d_val / abs_coef for d_val, abs_coef in zip(d_values, coefs_abs)]

        # Step 4: Partition into two subsets
        subset_1 = []
        subset_2 = []
        for i in range(m):
            if normalized_coefs[i] == 1:
                subset_1.append((normalized_coefs[i], normalized_d_values[i]))
            elif normalized_coefs[i] == -1:
                subset_2.append((normalized_coefs[i], normalized_d_values[i]))
        
        # Step 5: Calculate 𝑥 = max(𝑑1/coefs_1, ..., 𝑑𝑚/coefs_m) for Subset 1
        x_values = max(abs(normalized_d_values) for _, normalized_d_values in subset_1)#if normalized_d_values<=0)
        k = len(subset_1) if len(subset_1) > len(subset_2) else len(subset_2)
        attemps += 1

        solution_values = [x_values*r_values[i]*coefs_abs[i] for i in range(n)]
        
        num_ineqs = 0
        for (c_list, d) in zip(c_matrix, d_values):
            if sum([cj * xj for (cj, xj) in zip(c_list,solution_values)]) <= d+1E-3:
                num_ineqs = num_ineqs + 1

    print("attemps:", attemps) 

    return k, solution_values
    '''
    max_attempts = 10000000  # Set a maximum number of attempts
    attempts = 0
    max_satisfied = 0
    while (max_satisfied < (m/2)):
        # Generate random values for r1, r2, ..., rn
        r_values = [uniform(-1, 1) for _ in range(n)]
        
        # Substitute xi = ri * x for each variable xi
        substituted_inequalities = [
            [c_matrix[j][i] * r_values[i] for i in range(n)] for j in range(m)
        ]

        # Partition into two subsets based on coefficients
        subset_1 = [ineq for ineq in substituted_inequalities if any(coeff > 0 for coeff in ineq)]
        subset_2 = [ineq for ineq in substituted_inequalities if any(coeff < 0 for coeff in ineq)]

        # Solve Subset #1: x <= di
        max_subset_1 = max(max(subset) for subset in subset_1) if subset_1 else float('-inf')
        satisfied_subset_1 = sum(1 for d in d_values if max_subset_1 <= d)

        # Solve Subset #2: -x <= dj => x >= -dj
        min_subset_2 = min(min(subset) for subset in subset_2) if subset_2 else float('inf')
        satisfied_subset_2 = sum(1 for d in d_values if -min_subset_2 <= d)

        # Choose the subset with more satisfied inequalities
        max_satisfied = max(satisfied_subset_1, satisfied_subset_2)
        
        # Return the solution if it satisfies at least m/2 inequalities
        
        attempts += 1
        print("max_satisfied: ",max_satisfied)

    return max_satisfied, [r_values[i] for i in range(n)]  # Return k and r values

    '''
    # Compute x as the maximum value of r * x for each variable xi
    # Initialize subsets for positive and negative coefficient for x
    '''
    max_satisfied = 0
    solution_values = []
    while (max_satisfied < (np.ceil(m / 2) + 2)):
        # Generate random values for r1, r2, ..., rn
        r_values = [uniform(-1, 1) for _ in range(n)]
        
        # Compute x as the maximum value of r * x for each variable xi
        x = max(abs(d_values[j]) for i in range(n) for j in range(m))

        # Solve Subset #1: x <= di
        satisfied_subset_1 = [d for d in d_values if x <= d]

        # Solve Subset #2: -x <= dj => x >= -dj
        satisfied_subset_2 = [-d for d in d_values if x >= -d]

        # Choose the subset with more satisfied inequalities
        max_satisfied = max(len(satisfied_subset_1), len(satisfied_subset_2))
        
        # Determine the values of xi that satisfy the maximum number of inequalities
        solution_values = [r_values[i] * x for i in range(n)]
        
    print("max_satisfied:", max_satisfied)  
    return max_satisfied, solution_values  # Return k and solution values
    
    # No solution found satisfying m/2 constraints after max_attempts
    return 0, [None] * n
    '''

In [556]:
def testSolution(n, m, c_matrix, d_values, x_values):
    # always check pre-conditions: saves so much time later
    assert len(c_matrix) == m
    assert all(len(c_list) == n for c_list in c_matrix)
    assert len(d_values) == m
    assert len(x_values) == n
     # Check how many inequalities satisfied
    num_ineqs = 0
    for (c_list, d) in zip(c_matrix, d_values):
        if sum([cj * xj for (cj, xj) in zip(c_list,x_values )]) <= d+1E-3:
            num_ineqs = num_ineqs + 1
    assert num_ineqs >= m/2, f' Half number of inequalities to be sat: {m/2} your solution satisfies: {num_ineqs} inequalities '
    print('Test Passed')
    return 
        
        

n = 5
m = 24
c_matrix = [
    [1, -1, 0, 1, -1],
    [1, 2, 0, 0, 2],
    [-1, 0, 1, 1, 1],
    [1, 0, 0, 0, -1],
    [-1, 0, 0, -1, -1],
    [1, 0, 0, 1, 1],
    [1, 0, -1, 1, 0],
    [0, 2, 1, 0, 2],
    [-1, 1, 1, -1, 0],
    [1, 1, 1, 0, 1],
    [-1, 1, 1, 0, 0],
    [1, 1, 1, 1, 0],
    [-1, 1, 0, 1, -1],
    [1, -2, 0, 0, -2],
    [1, 0, 1, -1, -1],
    [1, 0, 1, 0, 1],
    [-1, 0, 0, 1, 1],
    [-1, 0, 0, 1, 1],
    [1, -1, 1, 1, 1],
    [0, -2, -1, 0, 2],
    [-1, -1, -1, -1, 0],
    [-1, 1, -1, 0, 1],
    [1, 0, 0, 1, 0],
    [-1, 0, -1, 0, -1],
]

d_list = [
    -5, 3, -4, -2, -3, -1,
    -5, 3, -4, -2, -3, -1,
     5, -3, 4, 2, 3, 1,
    5, -3, 4, 2, 3, 1,
    
]
(k, x_values) = computeApproximateSolution(n, m, c_matrix, d_list)
testSolution(n, m, c_matrix, d_list, x_values)
print('5 points')

attemps: 1
Test Passed
5 points


In [557]:
from random import uniform, randint, seed
## Warning: these are large instances. If your solution takes more than 120 seconds, then 
## chances are that you will not receive any credit for this problem.
def gen_random_instance(n, m):
    c_matrix = [ [randint(-5, 5) for i in range(n)] for j in range(m)]
    d_values = [randint(-10,10) for i in range(m)]
    return (c_matrix, d_values)

seed(100001)

print('Test # 1')
n = 10
m = 55
(c_matrix, d_values) = gen_random_instance(n, m)
(k, x_values) = computeApproximateSolution(n, m, c_matrix, d_values)
print(k)
print(x_values)
testSolution(n, m, c_matrix, d_values, x_values)


print('Test # 2')
n = 35
m = 230
(c_matrix, d_values) = gen_random_instance(n, m)
(k, x_values) = computeApproximateSolution(n, m, c_matrix, d_values)
print(k)
print(x_values)
testSolution(n, m, c_matrix, d_values, x_values)

print('Test # 3')
n = 100
m = 550
(c_matrix, d_values) = gen_random_instance(n, m)
(k, x_values) = computeApproximateSolution(n, m, c_matrix, d_values)
print(k)
print(x_values)
testSolution(n, m, c_matrix, d_values, x_values)

print('Test # 4')
n = 80
m = 900
(c_matrix, d_values) = gen_random_instance(n, m)
(k, x_values) = computeApproximateSolution(n, m, c_matrix, d_values)
print(k)
print(x_values)
testSolution(n, m, c_matrix, d_values, x_values)

print('Test # 5')
n = 70
m = 445
(c_matrix, d_values) = gen_random_instance(n, m)
(k, x_values) = computeApproximateSolution(n, m, c_matrix, d_values)
print(k)
print(x_values)
testSolution(n, m, c_matrix, d_values, x_values)

print('15 points!')

Test # 1
attemps: 1
30
[-59.34496672594772, -1.295904118618885, -81.64367969562595, -16.432994996102586, 29.970448596207167, -16.787364479453775, 80.89712761028014, 7.108159735432544, -53.06298279079206, -38.85921824232456]
Test Passed
Test # 2
attemps: 1
122
[-48.19043559651724, -28.46629366643628, -80.1953933658679, -34.48290703431231, 103.4558209535972, -116.2713997005794, 103.52646003697208, 126.30660353081187, -172.02461618846164, -68.42733342982802, 47.217089469133846, -136.114457970848, -400.8984016198072, 237.38785433231135, -291.07833696999353, 307.5809178960053, 19.249730999328936, 247.5590491585691, 199.9683266200419, 157.59869683143228, 305.51928092021194, 547.5735038719779, -523.2325550232364, 151.35523750405974, 47.55495830878167, 62.50656913095781, 145.43333421547706, -46.45033314527909, 108.81660074678716, -187.63429727186994, -480.1643532567256, 267.42385472857336, 128.67389263516202, 33.76641253802102, -188.91725764408457]
Test Passed
Test # 3
attemps: 2
281
[1134.028

## Selected Answers

### 2(A) part (i)

$$\max\ w_1 + \cdots + w_m $$

### 2(A) part (ii)

$ \sum_{i=1}^n c_i b_i $ where $b_i = \begin{cases} \ell_i & \text{if}\ c_i < 0 \\ 
u_i & \text{if}\  c_i \geq 0\\
\end{cases}$.

### 2(A) part (iii)
When $w_j = 1$, we have $\hat{I}_j$ is the inequality

$$ c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j \times 1 + M_j \times 0 = d_j $$

Therefore, $\hat{I}_j$ and $I_j$ coincide.

However, when $w_j = 0$, we have  $\hat{I}_j$ is the inequality

$$ c_{j1} x_1 + \cdots + c_{jn} x_n \leq d_j \times 0 + M_j \times 1 = M_j $$
However, given that $x_i \in [\ell_i, u_i]$, and $M_j = \mathsf{upperBoundLHS}(I_j)$,
it automatically implies that that $\hat{I}_j$ is satisfied trivially by whatever value of $(x_1, \ldots, x_n)$ we choose.

Partition the inequalities into two subsets: 

- All inequalities with +1 coefficient for $x$
$$ x \leq d_{i_1}, \ldots, x \leq d_{i_m} $$
By setting $x = \max(d_{i_1}, \ldots, d_{i_m})$ we can satisfy all these inequalities.

- All inequalities with a $-1$ coefficient for $x$
$$ - x \leq d_{j_1}, \ldots, -x \leq d_{j_m}$$

At least one of these partitions must account for $\lceil \frac{m}{2} \rceil$ or more constraints.

### That's all folks.